In [ ]:
using BSON
using BSON: @save, @load
using Flux
using Plots

In [ ]:
include("src/NN_Extender.jl")

In [ ]:
NN_Extender.main("model-pre-trained.bson")

In [ ]:
BSON.@load "model-pre-trained.bson" model
BSON.@load "output_model.bson" output_model

In [ ]:
NN_Extender.plot_model_parameters(model)

In [ ]:
NN_Extender.plot_model_parameters(output_model)

In [ ]:
input_vector1 = rand(10)
input_vector2 = [input_vector1; rand(); rand()]

In [ ]:
model(input_vector1)

In [ ]:
output_model(input_vector2)

# Testing

In [ ]:
mul = 10
for i in 1:100000
    input_vector1 = rand(10) .* mul
    input_vector2 = [input_vector1; rand() * mul; rand() * mul]
    result1 = model(input_vector1)
    result2 = output_model(input_vector2)
    if abs(result1[1] - result2[1]) >  0.00001
        println("Failed test")
        println("Difference: $(abs(result1[1] - result2[1]))")
    end
end